In [1]:
import numpy as np, os, sys, matplotlib.pyplot as plt, seaborn as sns, pandas, orjson
from tqdm.notebook import tqdm
from mosestokenizer import MosesTokenizer
from transformers import AutoTokenizer

tqdm.pandas()

import logging
logging.disable(logging.INFO)

In [2]:
def get_dict_of_df(df_dict, read_directory='../log_calculations/unified_outputs/', fname_to_read='filtered_df.csv'):
    
    dfs = {}
    read_dirs = []
    for exp, models in df_dict.items():
        for model in models:
            file_name = os.path.join(read_directory, exp, model, fname_to_read)
            read_dirs.append(os.path.join(read_directory, exp, model))
            dfs[model] = pandas.read_csv(file_name)
    return dfs, read_dirs

In [4]:
models = {
    # 'every_5_5000' : ['allenai/scibert_scivocab_cased', 'bert-base-cased', 'roberta-base', 'xlm-roberta-base']
    'every_5_5000' : ['bert-large-cased']
}

dfs, read_dirs = get_dict_of_df(models, fname_to_read='filtered_df_with_source.tsv')

In [5]:
languages = pandas.concat([df['langdetect'] for _, df in dfs.items()], axis=0).drop_duplicates()
languages

0           en
19          de
207         fr
212         es
286         nl
639         id
967         af
1427        ca
1497        pt
1758        no
1899        fi
2085        it
2154        ja
2284        ru
2565        pl
3752        sv
4650        ko
5472        cy
8316        cs
9155        tr
9172     zh-cn
12977       fa
13942       hr
16111       et
19208       vi
20703       ar
21548       sl
23840       bg
24489       hu
24919       uk
29960       ro
32891       da
35295       lv
37157    zh-tw
37908       sk
41657       sq
43624       tl
46012       el
49544       lt
49654       mk
52049       th
56578       so
66675       he
69065       hi
77635       sw
Name: langdetect, dtype: object

In [6]:
tokenizers = {}
for lang in languages:
    tokenizers[lang] = MosesTokenizer(lang)
    


In [7]:
for model, df in dfs.items():
    tokenizer = AutoTokenizer.from_pretrained(model)
    df['abstract_input'] = df['tokens'].progress_apply(lambda x: tokenizer.decode(eval(x)[1:-1]))
    df['num_moses_tokens'] = df[['langdetect', 'abstract_input']].progress_apply(lambda x: len(tokenizers[x.langdetect](x.abstract_input)), axis=1)

  0%|          | 0/98168 [00:00<?, ?it/s]

  0%|          | 0/98168 [00:00<?, ?it/s]

In [ ]:
for (model, df), dir in zip(dfs.items(), read_dirs):
    print(dir)
    print(model)
    df.to_csv(os.path.join(dir, 'filtered_df_with_moses_tokens.tsv'))

../log_calculations/unified_outputs/every_5_5000/bert-large-cased
bert-large-cased
